In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt
import matplotlib

matplotlib.style.use("ggplot")
from ipywidgets import interact
import numpy as np
import matplotlib as mpl
import arrow
from matplotlib import animation, rc
from IPython.display import HTML, display
from datetime import timedelta
import pandas_datareader.data as data


%matplotlib inline

# Lets take a stab at some stock analysis!

In [22]:
start_date = "2017-06-01"  # SNAP  IPO'd on 3/2/2017 - so start after that.
end_date = "2020-12-31"

# User pandas_reader.data.DataReader to load the desired data. As simple as that.
panel_data = data.DataReader(
    "FB;AMZN;SNAP;AAPL;QQQ;GOOG;MSFT;GOLD".split(";"), "yahoo", start_date, end_date
)
print("Sample from the full dataset")
display(panel_data.head(1))
df_original = panel_data["Close"]
print("Sample of data from close")
df = df_original.copy()
display(df.head(3))

Sample from the full dataset
Sample of data from close


Attributes   Adj Close                                                        \
Symbols             FB        AMZN   SNAP       AAPL         QQQ        GOOG   
Date                                                                           
2017-06-01  151.529999  995.950012  21.34  36.576199  137.745148  966.950012   

Attributes                             Close              ...       Open  \
Symbols          MSFT       GOLD          FB        AMZN  ...       MSFT   
Date                                                      ...              
2017-06-01  66.501236  15.553426  151.529999  995.950012  ...  70.239998   

Attributes                 Volume                                    \
Symbols          GOLD          FB       AMZN       SNAP        AAPL   
Date                                                                  
2017-06-01  16.309999  14530600.0  2454800.0  7146500.0  65616400.0   

Attributes                                                
Symbols            QQQ       GOOG        MSFT       GOLD  
Date                                                      
2017-06-01  20120700.0  1410500.0  21603600.0  9920100.0  

[1 rows x 48 columns]

Symbols,FB,AMZN,SNAP,AAPL,QQQ,GOOG,MSFT,GOLD
Date,,,,,,,,
2017-06-01,151.529999,995.950012,21.340000,38.294998,141.839996,966.950012,70.099998,16.240000
2017-06-02,153.610001,1006.729980,21.090000,38.862499,143.460007,975.599976,71.760002,16.200001
2017-06-05,153.630005,1011.340027,20.209999,38.482498,143.429993,983.679993,72.279999,16.190001


In [23]:
from numpy.lib.function_base import disp
import arrow

earliest = arrow.utcnow().shift(months=-12).date()
df = df_original.copy()[
    earliest:
]  # note this is destructive, probably good to keep an original around as a best practice

df.index = df.index.astype(
    str
)  # when transposing dates to columns, easier to operate in strings.

first_day = df.reset_index().iloc[0, 0]
last_day = df.reset_index().iloc[-1, 0]

display(f"Returns from:{first_day}, to:{last_day}")

returns = df.iloc[
    [0, -1]
].T  # first and last row, and turn into columns for easy manipulation
returns["delta"] = returns[last_day] - returns[first_day]
returns["pcnt_change"] = returns.delta / returns[first_day]
display(returns)

# returns.diff = returns.[] _[0] - _[1]
print(
    "QQ: Should sum daily %% change ==  pcnt_change of total returns -- because it doesn't??"
)
df.pct_change().sum()

'Returns from:2019-12-24, to:2020-12-24'

Date,2019-12-24,2020-12-24,delta,pcnt_change
Symbols,,,,
FB,205.119995,267.399994,62.279999,0.303627
AMZN,1789.209961,3172.689941,1383.479980,0.773235
SNAP,15.610000,50.150002,34.540002,2.212684
AAPL,71.067497,131.970001,60.902504,0.856967
QQQ,211.919998,309.559998,97.639999,0.460740
GOOG,1343.560059,1738.849976,395.289917,0.294211
MSFT,157.380005,222.750000,65.369995,0.415364
GOLD,18.309999,22.900000,4.590000,0.250683


QQ: Should sum daily %% change ==  pcnt_change of total returns -- because it doesn't??


Symbols
FB      0.370602
AMZN    0.647662
SNAP    1.445446
AAPL    0.728053
QQQ     0.443329
GOOG    0.331577
MSFT    0.444180
GOLD    0.340087
dtype: float64

In [26]:
from numpy.core.defchararray import encode
from altair.vegalite.v4.schema.channels import Tooltip

print("Correlations between stocks")
print("  NOTE: Need to correlate on percentage change, not abosolute price")


def display_correlation_matrix(df, earliest, latest):
    df = df_original.copy()
    df.columns.name = None
    corr = df.pct_change(1).corr()  # compute correlation on percent change
    df_dates = df[earliest:latest]
    corr = df_dates.corr().reset_index().melt(id_vars="index")
    # display(corr)
    height_in_inches = 40

    base = (
        alt.Chart(corr)
        .properties(
            width=8 * height_in_inches,
            height=8 * height_in_inches,
            title=f"Stock Correlation from: {earliest} to {latest}",
        )
        .encode(
            x="index:O",
            y="variable:O",
        )
    )

    c = base.mark_rect().encode(
        tooltip="value;index;variable".split(";"), color="value:Q"
    )
    text = base.mark_text(baseline="middle").encode(
        text=alt.Text("value:Q", format="0.2f")
    )
    display(c + text)


# Look at correlation
year_ago = arrow.utcnow().shift(months=-12).date()
today = arrow.utcnow().date()
latest = today
cv_19_start = "2020-02-01"

import datetime

first_day = df_original.reset_index().iloc[0, 0]

display_correlation_matrix(df, year_ago, today)
display_correlation_matrix(df, first_day, today)
display_correlation_matrix(df, "2020-06-01", today)
display_correlation_matrix(df, "2019-01-01", cv_19_start)
display_correlation_matrix(df, cv_19_start, today)

Correlations between stocks
  NOTE: Need to correlate on percentage change, not abosolute price


alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)